In [1]:
import csv
import random
from nnsight import LanguageModel

random.seed(10)

/home/local_nikhil/.conda/envs/mind/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Loading raw datasets

In [2]:
variable = "0_forward_belief"

with open(f"data/bigtom/{variable}_true_belief/stories.csv", "r") as f:
    reader = csv.reader(f, delimiter=";")
    tb_data = list(reader)

with open(f"data/bigtom/{variable}_false_belief/stories.csv", "r") as f:
    reader = csv.reader(f, delimiter=";")
    fb_data = list(reader)

with open(f"data/bigtom/{variable}_true_control/stories.csv", "r") as f:
    reader = csv.reader(f, delimiter=";")
    tb_control = list(reader)

with open(f"data/bigtom/{variable}_false_control/stories.csv", "r") as f:
    reader = csv.reader(f, delimiter=";")
    fb_control = list(reader)

# Loading model

In [11]:
model = LanguageModel("meta-llama/Meta-Llama-3-70B")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
priming_examples = """Rewrite the given original question to query about the ground truth regarding than anyone's perspective of the ground thruth. Use the following examples to generate an appropriate response.\nOriginal: Does Noor believe the milk pitcher contains oat milk or almond milk?\nResponse: Does the milk pitcher contains oat milk or almond milk?\n\nOriginal: Does Azola believe the package contains the book she ordered or a hand-knit scarf?\nResponse: Does the package contains the book she ordered or a hand-knit scarf?\n\nOrigial: Does Kavi believe there is a full bottle of hot sauce available or that the hot sauce has been spilled?\nResponse: Is there a full bottle of hot sauce available or that the hot sauce has been spilled?"""

In [12]:
story, question, correct_ans, wrong_ans = tb_data[random.randint(0, len(tb_data) - 1)]
prompt = f"{priming_examples}\n\nOriginal: {question}\nResponse:"

with model.generate("The Eiffel Tower is in the city of", remote=True):
    output = model.generator.output.save()
print("All prediction: ", model.tokenizer.batch_decode(output))

Setting `pad_token_id` to `eos_token_id`:FakeTensor(..., device='meta', size=(), dtype=torch.int64) for open-end generation.
`eos_token_id` should consist of positive integers, but is FakeTensor(..., device='meta', size=(1,), dtype=torch.int64). Your generation will not stop until the maximum length is reached. Depending on other flags, it may even crash.


GuardOnDataDependentSymNode: Could not guard on data-dependent expression u0 (unhinted: u0).  (Size-like symbols: none)

Potential framework code culprit (scroll up for full backtrace):
  File "/home/local_nikhil/.conda/envs/mind/lib/python3.10/site-packages/transformers/models/llama/modeling_llama.py", line 1250, in prepare_inputs_for_generation
    input_ids = input_ids[:, past_length:]

For more information, run with TORCH_LOGS="dynamic"
For extended logs when we create symbols, also add TORCHDYNAMO_EXTENDED_DEBUG_CREATE_SYMBOL="u0"
If you suspect the guard was triggered from C++, add TORCHDYNAMO_EXTENDED_DEBUG_CPP=1
For more debugging help, see https://docs.google.com/document/d/1HSuTTVvYH1pTew89Rtpeu84Ht3nQEFTYhAX3Ypa_xJs/edit?usp=sharing


In [97]:
def get_ground_truth_question(question):
    question = question.split(" ")
    new_question = ""
    if "is" in question:
        is_idx = question.index("is")
        del question[is_idx]
        believe_idx = question.index("believe")
        new_question += "Is the " + " ".join(question[believe_idx + 2:])
        return new_question
    else:
        del question[1], question[1]
        new_question = " ".join(question)
    
    return new_question

story, question, correct_ans, wrong_ans = tb_data[12]
print(f"Oringal: {question}")
print(f"New: {get_ground_truth_question(question)}")

Oringal: Does Kavi believe there is a full bottle of hot sauce available or that the hot sauce has been spilled?
New: Is the a full bottle of hot sauce available or that the hot sauce has been spilled?
